# **MDX 데이터 분석하기**
https://pypi.org/project/mdict-utils/

## **1 레시피 SQlite 데이터 불러오기**
작업한 내용을 대상으로 중분류 내용 추가하기
- 법률, 식품, 자동차 등 중분류 내용이 당장은 필요가 없어 보인다.
- 작업을 진행하면서 필요하면 추가하기

In [9]:
# 레시피 데이터베이스 불러오기
from muyong.nlp import mdx_to_df
df    = mdx_to_df('backup/recipe.db')
menus = df.entry.values.tolist()
len(menus), " ".join(menus)[:200]

(908,
 '가오리조림 가오리찜 가자미야채탕 가자미찜 가지냉채 가지불고기 가지선 가지오징어냉채 가지조림 가지회 각색두부전골 각색버섯덮밥 간장 갈비구이 갈비구이찜 갈비찜 갈비탕 감자가자미구이 감자갈비탕 감자고로케 감자그라땡 감자대구오븐구이 감자두부조림 감자막튀김 감자맑은국 감자밥 감자베이컨볶음 감자보트샐러드 감자볶음 감자볶음밥 감자볶음밥치즈구이 감자부침 감자새우그라땡 감')

In [10]:
food_recipes = df.paraphrase.values.tolist()
result = []
import re
for _ in food_recipes:
    result += re.findall("[가-힣]+", _)
len(result), len(list(set(result)))

88636

In [12]:
from konlpy.tag import Hannanum, Okt, Mecab, Komoran
food_text = " ".join(list(set(result)))
food_tokens = Mecab().nouns(food_text)
len(food_tokens)

9519

In [15]:
from nltk import Text
food_tokens_text = Text(food_tokens)
food_nouns = [_[0]  for _ in food_tokens_text.vocab().most_common()]
len(food_nouns)

2630

## **2 식재료 관련 데이터 찾기**
[전은경  맛을 표현하는 단어 (2003)](https://m.blog.naver.com/PostView.nhn?blogId=seongho0805&logNo=150048557298&proxyReferer=https%3A%2F%2Fwww.google.com%2F)

In [2]:
import pickle
with open('data/nerDict.pk', 'rb') as handle:
    nerDict = pickle.load(handle)

In [3]:
query     = "한천"  # 원문 내용의 확인
q_data    = nerDict[nerDict.Text == query].Data.values.tolist()
q_data

[[('명사', ['1', '몹시 가문 여름 하늘.']), ('명사', ['2', '몹시 가문 날씨.\r\n'])],
 [('명사',
   ['1', '겨울의 차가운 하늘.', '¶ 뿌옇게 흐렸던 한천에서 희끗희끗 눈발이 날리기 시작했다.≪이원규, 훈장과 굴레≫']),
  ('명사', ['2', ' =한절02.\r\n'])],
 [('명사', ['명사', '찬물이 솟는 샘.\r\n'])],
 [('명사', ['명사', '땀을 흘리며 헐떡거림.', '한천-하다01(汗喘--) [한ː---]']),
  ('동사', ['동사', '땀을 흘리며 헐떡거리다.\r\n'])],
 [('명사', ['명사', ' =우무01. ‘우무01’, ‘우뭇가사리’로 순화.\r\n'])],
 [('명사', ['명사', '『인명』', '‘이재04’의 호.\r\n'])]]

# **메뉴 데이터 N-Gram 분석**

## **1 N-Gram 데이터**
**itemIndexTemp** 를 활용하며 고유단어 찾기

In [18]:
# 명사 추출사전 데이터 호출
with open("data/nouns_food_03-2.txt", "r") as f:
    valid_token = f.read()
valid_token = valid_token.split(",")
len(valid_token)

693

In [19]:
# 분석할 메뉴데이터 호출
import pandas as pd  # 분석을 의뢰할 메뉴 데이터 [list]
menus_muyong  = pd.read_csv('data/menu_muyong.csv', encoding='ms949')
menus_muyong  = menus_muyong.dropna(subset=['메인1', '메인2'])
menus_muyong  = menus_muyong.fillna('')  # NaN 값을 지운다
tokens = []                # DataFrame 에서 Token List 추출
for _ in menus_muyong.columns[4:]:
    tokens += menus_muyong[_].values.tolist()

import re
from nltk import Text
menus_muyong = re.findall(r"[가-힣]+", " ".join(tokens))
menus_muyong = list(set(menus_muyong))
len(menus_muyong), " ".join(menus_muyong[:5])

(2880, '얼큰버섯전골 쥐어채볶음 매콤순대깻순볶음 돈안심꽈리장조림 짜장볶음우동')

In [20]:
# 메뉴 명사추출기
menu_token  = menus_muyong[1828]
print("원본내용:", menu_token)
from muyong.nlp import food_nouns
food_nouns(menu_token, valid_token)

원본내용: 쑥갓나물


['쑥갓나물']

## **2 만개의 레시피 활용**
긴 제목에서 레시피만 추출/ 나머지 구분하기

In [21]:
import pandas as pd
menus_man = pd.read_csv("data/1000recipe.csv", sep="|")
menus_man = menus_man.fillna('')
menus_man = menus_man.Menu.values.tolist()
menus_man[:3]

['호불호 없는 토마토파스타는 역시 ~! 토마토냉파스타 ★',
 '달콤 촉촉 달걀 푸딩 만들기, 만드는 법 ( 부드러운 달걀 요리 기본 원리 )',
 '손이가요 손이가~~자꾸 손이 가는 [가지전]']

In [22]:
# 메뉴 명사추출기
menu_token  = menus_man[4528]
print("원본내용:", menu_token)
from muyong.nlp import food_nouns
food_nouns(menu_token, valid_token)

원본내용: 아이볶음밥 김치볶음밥 카레가루를 넣어봐요!


['볶음밥', '김치', '카레']

# **Unique Token 묶음 분석**
메뉴의 이름들 중 대표성 높은 메뉴들 찾기

## **1 레시피 데이터 특징찾기**
잡일보단 **머신러닝 분석** 으로 우선 **모델** 만들기 (30일 발표용)
1. 구내식당은 대략 **2,300 개로** 추출 된다
1. 만개의 레시피는 50,000개로 특징이 추출
1. 우선은 **2,300 개** 로 **모델 사이트를 우선 만들어서 9월초 시연모델** 만들기

In [23]:
# # 10,000 개 레시피 분석
# result = ["_".join(sorted(food_nouns(menu, valid_token)))  
#           for menu in menus_man  
#           if "_".join(sorted(food_nouns(menu, valid_token)))]

# result = list(sorted(set(result)))
# len(menus_man), len(result)

# Muyong 레시피 분석
result = ["_".join(sorted(food_nouns(menu, valid_token)))  
          for menu in menus_muyong  
          if "_".join(sorted(food_nouns(menu, valid_token)))]

result = list(sorted(set(result)))
len(menus_muyong), len(result)

(2880, 2335)

In [10]:
with open("menus_unique_token.txt", 'w') as f:
    f.write(",".join(result))
print("file saved")

file saved


## **2 네이버 레시피 데이터 수집하기**
특징을 대상으로 네이버 블로그 데이터 대상 수집하기
1. 저장결과를 Sqlite3 로 저장하기
1. 안정적인 작업을 수행하도록 잘 구성하기

In [17]:
%%time
import os, sqlite3, json, time
con = sqlite3.connect("backup/foods.db")

import pandas as pd
df         = pd.read_sql("select * from 'foods'", con, index_col=None).drop('index', axis=1)
con.close()
food_links = df.data.values.tolist()
df.data    = [json.loads(_) for _ in food_links] # json 디코딩
df.head(2)

CPU times: user 14.1 s, sys: 1.17 s, total: 15.2 s
Wall time: 15.2 s


,title,data
0,가슴살_계란_닭_장조림,[{'title': '부드러운 닭<b>가슴살</b> <b>장조림</b> <b>만들기...
1,가슴살_굴소스_닭_볶음밥,[{'title': '수비드 닭<b>가슴살</b> 요리 : 돈까스 &amp; <b>...


In [26]:
# 수집값이 없는 인덱스를 새로 수정하기 
# Token Index 값이 유용한지를 Naver OpenAPI 결과값으로 구분하는 방법도 유용할 듯!!
index_err = [no  for no,_ in enumerate(df.data.values.tolist())  if len(_) == 0]
token_temp_list = ["가쓰오_어묵_유부","가쓰오_유부","가쓰오_유부_팽이","감자_미트볼_알감자_조림",\
              "돈육_불고기_간장","떡_부추_해물","무침_부추_콩나물","부추_전_해물","오징어_탕수"]

for no, i in enumerate(index_err):
    df.iloc[i, [0]] = token_temp_list[no]

In [29]:
%%time
from muyong.naver import get_blog_list
from tqdm import tqdm
result = {}
for _ in tqdm(token_temp_list):
    temp = []
    query_token  = _+"_만들기"
    for i in range(1, 500, 100):
        temp += get_blog_list(query_token, i)['items']
    time.sleep(.3)
    result[_] = temp

100%|██████████| 9/9 [00:17<00:00,  2.00s/it]

CPU times: user 661 ms, sys: 98 ms, total: 759 ms
Wall time: 17.7 s


In [50]:
for no, i in enumerate(index_err):
    df.iloc[i, 1] = result[token_temp_list[no]]

In [55]:
import pickle
with open('data/naver_key_list.pk', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("save is done")

save is done


In [6]:
with open("menus_unique_token.txt", "r") as f:
    word_tokens = f.read().split(",")

# get_blog_list("가슴살_닭_매콤_볶음_우동"+"_만들기", 101)["items"][:2]
# len(word_tokens), word_tokens[1000]

In [10]:
list(result.keys())

['가쓰오_어묵_유부',
 '가쓰오_유부',
 '가쓰오_유부_팽이',
 '감자_미트볼_알감자_조림',
 '돈육_불고기_간장',
 '떡_부추_해물',
 '무침_부추_콩나물',
 '부추_전_해물',
 '오징어_탕수']

In [11]:
import pandas as pd

In [14]:
len(result[list(result.keys())[0]])

500

In [1]:
%%time
import pandas as pd
from tqdm import tqdm
from muyong.naver import get_blog_list
import time, json, os, sqlite3
file_db = "foods.db"

token_temp_list = ["가쓰오_어묵_유부","가쓰오_유부","가쓰오_유부_팽이","감자_미트볼_알감자_조림",\
              "돈육_불고기_간장","떡_부추_해물","무침_부추_콩나물","부추_전_해물","오징어_탕수"]

for _ in tqdm(token_temp_list):
    # web link 목록 수집하기
    result, temp = [], []
    query_token  = _+"_만들기"
    for i in range(1, 500, 100):
        temp += get_blog_list(query_token, i)['items']
    time.sleep(.3)
    result.append([_, temp])
    
    # sqlite3 저장하기 (단어별 결과를 Json으로 저장 (list는 저장시 오류))
    df  = pd.DataFrame(result)
    df.columns = ['title', 'data']
    df.data    = [ json.dumps(_)  for _ in df.data ]
    con = sqlite3.connect(file_db)
    df.to_sql('foods', con, if_exists='append')
    con.commit()
    time.sleep(.1)

100%|██████████| 9/9 [00:19<00:00,  2.16s/it]

CPU times: user 1.43 s, sys: 464 ms, total: 1.9 s
Wall time: 20 s


In [2]:
df

,title,data
0,오징어_탕수,"[{""title"": ""[\ub3c4\uc2dd\ub2f9]<b>\ud0d5\uc21..."


## **3 링크 url 데이터만 저장하기**
**API 수집결과 중 Link Url** 만 저장하기

In [1]:
import pickle
with open('backup/naver_api_org.pk', 'rb') as handle:
    df = pickle.load(handle)

# data 에서 naver 링크만 찾아서 정리하기
df.head(3)

,title,data
0,가슴살_계란_닭_장조림,[{'title': '부드러운 닭<b>가슴살</b> <b>장조림</b> <b>만들기...
1,가슴살_굴소스_닭_볶음밥,[{'title': '수비드 닭<b>가슴살</b> 요리 : 돈까스 &amp; <b>...
2,가슴살_닭_라이스_카레,[{'title': '닭의품격 초간편 냉동닭<b>가슴살</b> <b>카레</b><b...


In [2]:
result = []
from tqdm import tqdm
for datum in tqdm(df.data.values.tolist()):
    temp = []
    for _ in datum:  # 500개 개별확인
        if _['link'].find("https://blog.naver.com/") != -1:
            temp.append(_['link'])
    result.append("|".join(temp))
df.data = result
df.to_csv('data/naver_links.csv', index=None)
df.head()

100%|██████████| 2335/2335 [00:00<00:00, 4736.39it/s]


,title,data
0,가슴살_계란_닭_장조림,https://blog.naver.com/berry__chu?Redirect=Log...
1,가슴살_굴소스_닭_볶음밥,https://blog.naver.com/happyday82?Redirect=Log...
2,가슴살_닭_라이스_카레,https://blog.naver.com/hogengggy?Redirect=Log&...
3,가슴살_닭_매콤_볶음_우동,https://blog.naver.com/cutylee82?Redirect=Log&...
4,가슴살_닭_볶음밥_파인애플,https://blog.naver.com/aurelly3280?Redirect=Lo...


In [ ]:
# Today
# 단어 Token 을 나열된 순서대로 정리하여 문법구조 분석 알고리즘 적용
# CFG 와 유사한 방식으로 불필요한 단어 및 묶음 정리하기
# 묶음 기준을 설정한 뒤, 크롤링을 통해 단어별 문서 300개를 수집 분석하여 연관단어 찾기

명엽채(명태채)/청포북(녹두묵)

1. 석박지/무석박지

```
마요 => 마요네즈
소시지 => 소세지
꺳잎 => 깻잎
게란 => 계란
배춧 => 배추
불낙 => 낙지
콘 => 옥수수
돈 => 돼지
탕수 => 탕수육
우무 => 우묵
중국식/
한국식/
```

In [ ]:
import pickle
with open('data/nerDict.pk', 'wb') as handle:
    pickle.dump(nerDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('data/nerDict.pk', 'rb') as handle:
    nerDict = pickle.load(handle)

# **Naver Post Detail Crawling**
메뉴의 이름들 중 대표성 높은 메뉴들 찾기

## **1 Naver Link 목록**
OpenAPI 로 수집한 결과값 저장된 목록 List 원본의 수정보완
1. Naver OPEN API 결과 **Token 이 유효값인지** 확인 가능했다 (Oh!!)
1. 네이버 블로그 목록만 정리한 DataBase 만들기
1. 목록을 4개로 나워서 2개의 서버에서 돌리며 저장하기

In [1]:
import re
# 수집결과 불필요한 기호들 제거하는 함수
def blog_preprocess(post):
    post_temp = post.replace('\n',"").replace('\t',"").split(".")
    post_temp = [" ".join(re.findall(r"[가-힣0-9A-z]+", _))  for _ in post_temp  if len(_.strip()) >= 1]
    return ".".join(post_temp)

In [ ]:
import pandas as pd
links_org      = pd.read_csv('data/naver_links.csv')
# links_org.shape, links_org.head(3)
links_by_token = links_org.iloc[::60, :] # 길이를 절반으로 축소
links_by_token = links_by_token.data.values.tolist()
len(links_by_token), links_by_token[0][:40]

In [ ]:
import sqlite3, time, os, requests_cache
file_db = "naver.db"
conn = sqlite3.connect(file_db)
requests_cache.install_cache('cache')

import pandas as pd
from tqdm import tqdm
from muyong.util import Telegram
from muyong.naver import get_blog_post
t = Telegram()
links_length = len(links_by_token)
for no, links in tqdm(enumerate(links_by_token)):
    result, temp = [], []
    links = links.split('|')
    for _ in links[:100]:
        _ = get_blog_post(_)
        _ = blog_preprocess(_)
        temp.append(_)
        time.sleep(.1)
    temp = "|".join(temp)     # 500개를 저장
    result.append([no, temp])
    df = pd.DataFrame(result)
    df.to_sql('foods', conn, if_exists='append')
    os.remove("cache.sqlite")
    if no % 200 == 0:
        t.msg("{}/ {} th processing".format(no, links_length))
    time.sleep(.1)
conn.commit()
conn.close()

## **2 수집결과 살펴보기**
Sqlite 결과를 보다 활용하기 쉽게 저장하기

In [ ]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("naver.db")
df = pd.read_sql('SELECT * FROM "foods"', conn, index_col=None).drop('index', axis=1)
conn.close()
df = df.iloc[11:].reset_index(drop=True)
df.columns = ['Query', 'Posts']
df.Query = links_org.iloc[::60, 0].tolist()
df.head()

In [ ]:
df.to_csv("data/naver_posts_sample.csv", index=None)

In [ ]:
df.Query.tolist()